# Homework 01: Numerical python and data handling

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
import os.path
import subprocess

In [8]:
def wget_data(url):
    local_path='./tmp_data'
    subprocess.run(["wget", "-nc", "-P", local_path, url])

In [9]:
def locate_data(name, check_exists=True):
    local_path='./tmp_data'
    path = os.path.join(local_path, name)
    if check_exists and not os.path.exists(path):
        raise RuntimeError('No such data file: {}'.format(path))
    return path

## Problem 1

Use `np.einsum` to evaluate the tensor expression $g^{il} \Gamma^m_{ki} x^k$ which arises in [contravariant derivatives in General Relativity](https://en.wikipedia.org/wiki/Christoffel_symbols#Covariant_derivatives_of_tensors).  Note we are using the GR convention that repeated indices (k,l) are summed over.

In [10]:
def tensor_expr(g, Gamma, x, D=4):
    """Evaluate the tensor expression above.

    Parameters
    ----------
    g : array
        Numpy array of shape (D, D)
    Gamma : array
        Numpy array of shape (D, D, D)
    x : array
        Numpy array of shape (D,)
    D : int
        Dimension of input tensors.

    Returns
    -------
    array
        Numpy array of shape (D, D) that evaluates the tensor expression.
    """
    assert g.shape == (D, D)
    assert Gamma.shape == (D, D, D)
    assert x.shape == (D,)

    # YOUR CODE HERE
    return np.einsum('il, mki, k -> lm', g, Gamma, x)

In [11]:
# A correct solution should pass these tests.
g = np.arange(4 ** 2).reshape(4, 4)
Gamma = np.arange(4 ** 3).reshape(4, 4, 4)
x = np.arange(4)
y = tensor_expr(g, Gamma, x)
print(y)
assert np.array_equal(
    y,
    [[ 1680,  3984,  6288,  8592], [ 1940,  4628,  7316, 10004],
     [ 2200,  5272,  8344, 11416], [ 2460,  5916,  9372, 12828]])

[[ 1680  3984  6288  8592]
 [ 1940  4628  7316 10004]
 [ 2200  5272  8344 11416]
 [ 2460  5916  9372 12828]]


## Problem 2

Use `np.histogram` to calculate the fraction of values in an arbitrary input data array that lie in each of the 10 intervals \[0.0, 0.1), \[0.1, 0.2), ..., \[0.9, 1.0). You can assume that all input values are in the range \[0,1). This is a useful technique to estimate the probability density that the data was sampled from.

In [12]:
def estimate_probability_density(data, bins):
    """Estimate the probability density of arbitrary data.

    Parameters
    ----------
    data : array
        1D numpy array of random values.
    bins : array
        1D numpy array of N+1 bin edges to use. Must be increasing.

    Returns
    -------
    array
        1D numpy array of N probability densities.
    """
    assert np.all(np.diff(bins) > 0)

    # YOUR CODE HERE
    hist, bin_edges = np.histogram(data,bins)
    print((np.roll(bin_edges, -1) - bin_edges))
    bin_widths = np.array(np.roll(bin_edges, -1) - bin_edges)[:len(hist)]
    probability = hist/len(data)/bin_widths
    return probability

In [13]:
# A correct solution should pass these tests.
generator = np.random.RandomState(seed=123)
data = generator.uniform(size=100)
bins = np.linspace(0., 1., 11)
rho = estimate_probability_density(data, bins)
print(rho)
assert np.allclose(0.1 * rho.sum(), 1.)
assert np.allclose(rho, [ 0.6,  0.8,  0.7,  1.7,  1.1,  1.3,  1.6,  0.9,  0.8,  0.5])

[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1 -1. ]
[0.6 0.8 0.7 1.7 1.1 1.3 1.6 0.9 0.8 0.5]


## Problem 3

Define a function to calculate the [entropy](https://en.wikipedia.org/wiki/Entropy_estimation) $H(\rho)$ of a binned probability density, defined as:
$$
H(\rho) \equiv -\sum_i \rho_i \log(\rho_i) \Delta w_i \; ,
$$
where $\rho_i$ is the binned density in bin $i$ with width $w_i$.

In [14]:
def binned_entropy(rho, bins):
    """Calculate the binned entropy.

    Parameters
    ----------
    rho : array
        1D numpy array of densities, e.g., calculated by the previous function.
    bins : array
        1D numpy array of N+1 bin edges to use. Must be increasing.

    Returns
    -------
    float
        Value of the binned entropy.
    """
    assert np.all(np.diff(bins) > 0)

    # YOUR CODE HERE
    bin_widths = np.array(np.roll(bins, -1) - bins)[:len(rho)]
    print(bin_widths)
    entropy = np.sum(rho*np.log(rho)*bin_widths)
    print(entropy)
    return(-entropy)

In [15]:

# A correct solution should pass these tests.
generator = np.random.RandomState(seed=123)
data1 = generator.uniform(size=10000)
data2 = generator.uniform(size=10000) ** 4
bins = np.linspace(0., 1., 11)
rho1 = estimate_probability_density(data1, bins)
rho2 = estimate_probability_density(data2, bins)
H1 = binned_entropy(rho1, bins)
H2 = binned_entropy(rho2, bins)
assert np.allclose(H1, -0.000801544)
assert np.allclose(H2, -0.699349908)

[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1 -1. ]
[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1 -1. ]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
0.0008015440491370906
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
0.699349908506402


## Problem 4

Define a function that reads `pong_data.hf5` and returns a new subset DataFrame containing only the columns `x5`, `y5`, `x7`, `y7` (**in that order**) and only the last 200 rows.

In [16]:
wget_data('https://courses.physics.illinois.edu/phys503/fa2023/data/pong_data.hf5')
pong_data = pd.read_hdf(locate_data('pong_data.hf5'))
print(pong_data)

           x0        x1        x2        x3        x4        x5        x6  \
0   -0.065289  0.026892  0.116548  0.203748  0.288559  0.371047  0.451275   
1    0.104788  0.170583  0.234577  0.296817  0.357352  0.416228  0.473492   
2    0.034411  0.114257  0.191914  0.267445  0.340906  0.412355  0.481846   
3    0.018970  0.092728  0.164466  0.234239  0.302100  0.368102  0.432295   
4    0.102970  0.168359  0.231957  0.293812  0.353973  0.412486  0.469395   
..        ...       ...       ...       ...       ...       ...       ...   
995  0.098345  0.148411  0.197106  0.244466  0.290529  0.335330  0.378904   
996  0.097067  0.150260  0.201996  0.252315  0.301255  0.348855  0.395150   
997 -0.161553 -0.089041 -0.018516  0.050077  0.116790  0.181677  0.244785   
998 -0.006522  0.075279  0.154840  0.232221  0.307482  0.380682  0.451876   
999 -0.027545  0.058443  0.142074  0.223415  0.302527  0.379472  0.454309   

           x7        x8        x9   y0        y1        y2        y3  \
0  

In [74]:
def create_subset():
    """Read pong_data.hf5 and return a subset.
    """
    # YOUR CODE HERE
    columns = pong_data[['x5','y5', 'x7', 'y7']]
    subset = columns [-200:]
    return subset

In [75]:
print(create_subset())

           x5        y5        x7        y7
800  0.345902  0.285369  0.502691  0.185569
801  0.386747  0.315539  0.525772  0.226682
802  0.510889  0.282691  0.669121  0.181920
803  0.393058  0.280415  0.552502  0.178819
804  0.575114  0.237782  0.755070  0.120723
..        ...       ...       ...       ...
995  0.335330  0.379540  0.421284  0.313895
996  0.348855  0.374642  0.440177  0.307221
997  0.181677  0.336711  0.306165  0.255533
998  0.380682  0.313319  0.521120  0.223656
999  0.379472  0.301558  0.527096  0.207630

[200 rows x 4 columns]


In [76]:
# A correct solution should pass these tests.
subset = create_subset()
assert np.array_equal(subset.columns.values, ('x5', 'y5', 'x7', 'y7'))
assert len(subset) == 200
summary = subset.describe()
assert np.allclose(summary.loc['mean', :].values,
                   [ 0.43564752,  0.30610958,  0.57520991,  0.21383226])